In [2]:
import os
import sys
import requests
from importlib import reload
import json
from datetime import datetime
import pandas as pd

In [3]:
sys.path.append('..') #agregar directorio madre del repo para poder importar librerias customizadas
#importar librerias customizadas

In [4]:
import ETL.Elastic_Module
reload(ETL.Elastic_Module)

<module 'ETL.Elastic_Module' from '../ETL/Elastic_Module.py'>

In [5]:
from ETL.Elastic_Module import *

In [6]:
import kibana_backup
reload(kibana_backup)

<module 'kibana_backup' from '/home/jovyan/work/backup_kibana/kibana_backup.py'>

In [7]:
from kibana_backup import *

In [8]:
args={}
args['elastic_user']='elastic'
args['elastic_psswd']=os.environ['ELASTIC_PASSWD']
args['kibana_url']='https://staging.kibana.wenuwork.cl'

In [9]:
#Cargar backup de Kibana
path='backups/'
file='kibana_bckp05_13_2020__06_30_44.json'
with open(path+file, 'r') as fp:
    back=json.load(fp)

In [10]:
back=backup(args['kibana_url'], args['elastic_user'], args['elastic_psswd'],return_type='json')

In [11]:
back.keys()

dict_keys(['visualization', 'dashboard', 'search', 'index-pattern', 'config', 'timelion-sheet'])

In [12]:
back['config']

{'page': 1,
 'per_page': 20,
 'total': 1,
 'saved_objects': [{'type': 'config',
   'id': '7.2.0',
   'attributes': {'buildNum': 24337,
    'defaultIndex': 'd3d7af60-4c81-11e8-b3d7-01146121b73d',
    'theme:darkMode': True,
    'fields:popularLimit': 50,
    'courier:ignoreFilterIfFieldNotInIndex': True,
    'dateFormat:tz': 'UTC'},
   'references': [],
   'updated_at': '2020-05-14T23:40:43.022Z',
   'version': 'WzMyMyw0Ml0='}]}

In [23]:
ipy=backup(args['kibana_url'], args['elastic_user'], args['elastic_psswd'],return_type='json',saved_object_types=('index-pattern',))
ipy=get_index_patterns(ipy) #index patterns del host o target

In [24]:
ipy

,id,title
0,95fc7190-7ed3-11ea-ab0e-9d817504fbf0,kibana_sample_data_flights*
1,01319e90-7ede-11ea-ab0e-9d817504fbf0,factor_emision
2,ba61c5b0-7a7a-11ea-ab0e-9d817504fbf0,test_import
3,90096c60-7ef7-11ea-ab0e-9d817504fbf0,test_import_fe
4,fef02360-80e8-11ea-ab0e-9d817504fbf0,test_reporte_radar
5,7a529ce0-8215-11ea-ab0e-9d817504fbf0,instalaciones
6,70ffa740-77da-11ea-ab0e-9d817504fbf0,sensores
7,4c8f8130-933e-11ea-9b23-69d084343f32,es_errors
8,540b29e0-9425-11ea-9b23-69d084343f32,monitors
9,8b1a5fb0-9582-11ea-9b23-69d084343f32,stats_company_hour


In [54]:
ipx=get_index_patterns(back) #index patterns del respaldo

In [55]:
ipx

,id,title
0,d1ff6560-56d5-11ea-a1c5-ed7c7dd23648,stats_company_hour
1,955a83c0-56e8-11ea-a1c5-ed7c7dd23648,monitors_pivot


In [38]:
replaced=change_index_pattern_references(back,types=['visualization', 'dashboard'],mapping=mapping)

In [42]:
change_viz_fields(replaced,mapping=mapping)

{'visualization': {'page': 1,
  'per_page': 20,
  'total': 45,
  'saved_objects': [{'type': 'visualization',
    'id': '71ffc010-7a75-11ea-ab0e-9d817504fbf0',
    'attributes': {'title': '[test] ejemplo',
     'visState': '{"title":"[test] ejemplo","type":"line","params":{"type":"line","grid":{"categoryLines":false},"categoryAxes":[{"id":"CategoryAxis-1","type":"category","position":"bottom","show":true,"style":{},"scale":{"type":"linear"},"labels":{"show":true,"truncate":100},"title":{}}],"valueAxes":[{"id":"ValueAxis-1","name":"LeftAxis-1","type":"value","position":"left","show":true,"style":{},"scale":{"type":"linear","mode":"normal"},"labels":{"show":true,"rotate":0,"filter":false,"truncate":100},"title":{"text":"Promedio Factor Emisión"}}],"seriesParams":[{"show":"true","type":"line","mode":"normal","data":{"label":"Promedio Factor Emisión","id":"1"},"valueAxis":"ValueAxis-1","drawLinesBetweenPoints":true,"showCircles":true}],"addTooltip":true,"addLegend":true,"legendPosition":"ri

In [56]:
#mapping de IDs de index patterns X-->Y
mapping=ipx.merge(ipy,on='title',how='left')
mapping=dict(zip(mapping.id_x, mapping.id_y))

In [57]:
mapping

{'d1ff6560-56d5-11ea-a1c5-ed7c7dd23648': '8b1a5fb0-9582-11ea-9b23-69d084343f32',
 '955a83c0-56e8-11ea-a1c5-ed7c7dd23648': '98bbc0f0-9582-11ea-9b23-69d084343f32'}

In [59]:
#Guardar mapping de index patterns
path='parameters/'
now = datetime.now() # current date and time
date_time = now.strftime("%m_%d_%Y__%H_%M_%S")
with open(path+'_index_pattern_mapping_'+date_time+'.json', 'w') as fp:
    json.dump(mapping, fp)

In [37]:
list_viz(back)

,title,id,index
0,[test] ejemplo,71ffc010-7a75-11ea-ab0e-9d817504fbf0,0
1,[TEST] Factor Emisión F.Humeres,f37e2a00-7edf-11ea-ab0e-9d817504fbf0,1
2,[TEST] Tabla Factor Emisión F.Humeres,1b55f390-7ef0-11ea-ab0e-9d817504fbf0,1
3,test - viz sensores,4c3a3a10-7a78-11ea-ab0e-9d817504fbf0,1
4,[Test 2] Factor emisión F.Humeres,1a4a7450-801f-11ea-ab0e-9d817504fbf0,1
5,[Wenu Sensores] - Listado,1c4393d0-77dd-11ea-ab0e-9d817504fbf0,1
6,[Wenu instalaciones] - Listado corregido,a05846e0-8217-11ea-ab0e-9d817504fbf0,1
7,[Wenu Sensors] - Controles,30cb8230-77de-11ea-ab0e-9d817504fbf0,1
8,[Wenu Errores] - ES Errors,9a0844b0-933e-11ea-9b23-69d084343f32,1
9,[Flights] Controls,aeb212e0-4c84-11e8-b3d7-01146121b73d,1


In [43]:
back['visualization']['saved_objects'][0]

{'type': 'visualization',
 'id': '71ffc010-7a75-11ea-ab0e-9d817504fbf0',
 'attributes': {'title': '[test] ejemplo',
  'visState': '{"title":"[test] ejemplo","type":"line","params":{"type":"line","grid":{"categoryLines":false},"categoryAxes":[{"id":"CategoryAxis-1","type":"category","position":"bottom","show":true,"style":{},"scale":{"type":"linear"},"labels":{"show":true,"truncate":100},"title":{}}],"valueAxes":[{"id":"ValueAxis-1","name":"LeftAxis-1","type":"value","position":"left","show":true,"style":{},"scale":{"type":"linear","mode":"normal"},"labels":{"show":true,"rotate":0,"filter":false,"truncate":100},"title":{"text":"Promedio Factor Emisión"}}],"seriesParams":[{"show":"true","type":"line","mode":"normal","data":{"label":"Promedio Factor Emisión","id":"1"},"valueAxis":"ValueAxis-1","drawLinesBetweenPoints":true,"showCircles":true}],"addTooltip":true,"addLegend":true,"legendPosition":"right","times":[],"addTimeMarker":false,"dimensions":{"x":{"accessor":0,"format":{"id":"terms"

In [46]:
mapping_fields={'sub_location':'subLocation_label'}

In [58]:
#Guardar mapping de campos en visualizaciones
path='parameters/'
now = datetime.now() # current date and time
date_time = now.strftime("%m_%d_%Y__%H_%M_%S")
with open(path+'_viz_field_mapping_'+date_time+'.json', 'w') as fp:
    json.dump(mapping_fields, fp)